<a href="https://colab.research.google.com/github/BorisMejiasII/Hierarchical-Clustering/blob/main/Notebook_Tarea_Zoo_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering Jerárquico en el Dataset Zoo
Este notebook es una base para la tarea de clustering jerárquico usando el dataset Zoo.

In [ ]:
# Paso 1: Cargar el dataset


In [ ]:
# Paso 2: Preprocesamiento (One Hot Encoding y Escalamiento)


In [ ]:
# Paso 3: Clustering Jerárquico


In [ ]:
# Paso 4: Heatmap


# Conclusiones
Escribe aquí tus conclusiones sobre las agrupaciones obtenidas en el clustering jerárquico.

## Pregunta 1
1. ¿Cuántos clusters se pueden identificar claramente en el dendrograma?

Escribe tu respuesta aquí.

## Pregunta 2
2. ¿Qué características parecen determinar la formación de los clusters?

Escribe tu respuesta aquí.

## Pregunta 3
3. ¿Los resultados del clustering tienen sentido biológico o zoológico?

Escribe tu respuesta aquí.

## Pregunta 4
4. ¿Cómo afecta la normalización de los datos al clustering?

Escribe tu respuesta aquí.

## Pregunta 5
5. ¿Qué método de linkage utilizaste y por qué?

Escribe tu respuesta aquí.

## Pregunta 6
6. ¿Qué otras técnicas de clustering podrías aplicar a este dataset y por qué?

Escribe tu respuesta aquí.